### agent? 
- 우리의 대리인이라 생각하면 됨
- 사용자가 명령을 주면, (tool) 알아서 검토하고 작업 결과물을 전달함.
- chain of thought 에 행동을 부여

#### agent를 사용하기 위한 세가지 
- 기본 LLM
- tool
- control the interaction agent 

#### - agent 작동 방식 
1. input 할당 (사용자가)
2. agent가 생각
3. action/action input 사용할 도구 tool 결정, 도구의 input 결정
4. 도구의 출력 결과 관찰 observation
5. 관찰 결과가 작업이 완료 됐다고 판단될 때까지 과정을 반복함.

ex. 계산기
- llmmathchain을 사용하여 계산하는 함수를 만들어서 사용할거임 = 초기화함
- tool 생성함, 이 tool엔 llm_math.run 가 실행하도록 저장되어 있음.
- 질문이 들어오면, 질문이 tool 리스트에 들어가서 llm_math.run 함수를 거쳐 계산을 해주도록 함 

### 일반 에이전트
- 다양한 도구를 사용하여 작업을 수행
- 각 도구는 특정 작업을 처리하며, 에이전트는 적절한 도구를 선택하여 실행
### ReAct 프레임워크
- 에이전트가 추론과 행동을 결합하여 복잡한 작업을 단계적으로 해결, 더 정교한 문제 해결이 필요할 때 유용
- 모델이 단순히 질문에 대한 즉각적인 답변을 제공하는 것을 넘어서, 문제를 단계별로 분석하고 중간 결과를 바탕으로 최종 결론을 도출하는 방식을 채택

In [1]:
import os
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

In [2]:
from langchain import OpenAI

llm = OpenAI(
    openai_api_key = "OPENAI_API_KEY",
    temperature = 0,
    model_name = "text-davinci-003"
)

C:\Users\82103\anaconda3\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [3]:
from langchain.chains import LLMMathChain
from langchain.agents import Tool

llm_math = LLMMathChain(llm=llm)

# initialize the math tool
math_tool = Tool(
    name='Calculator',
    func=llm_math.run,
    description='Useful for when you need to answer questions about math.'
)
# when giving tools to LLM, we must pass as list of tools
tools = [math_tool]

C:\Users\82103\anaconda3\lib\site-packages\langchain\chains\llm_math\base.py:57: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [4]:
from langchain.agents import load_tools

tools = load_tools(
    ['llm-math'],
    llm=llm
)

In [5]:
tools[0].name, tools[0].description

('Calculator', 'Useful for when you need to answer questions about math.')

In [6]:
from langchain.agents import initialize_agent

zero_shot_agent = initialize_agent(
    agent = "zero-shot-react-description",
    tools = tools,
    llm = llm,
    verbose = True,
    max_iterations = 3
)

C:\Users\82103\anaconda3\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [7]:
zero_shot_agent("what is (4.5*2.1)^2.2?")

C:\Users\82103\anaconda3\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: OPENAI_A**_KEY. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [8]:
zero_shot_agent("if Mary has four apples and Giorgio brings two and a half apple "
                "boxes (apple box contains eight apples), how many apples do we "
                "have?")



> Entering new AgentExecutor chain...


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: OPENAI_A**_KEY. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

- tool 리스트 목록에 여러 가지를 추가할 수 있음.
- 그래서 agent가 tool 목록에서 적절한 걸 선택해 실행함. 

### zeroshot react
ReAct 프롬프팅? 추론과 행동의 결합
명시적 추론 및 행동 관리 / 복잡한 문제 해결 / *에이전트의 투명성 향상
* agent는 각 도구를 사용하는 과정이 명시적으로 관리되지 않음 

ReAct: agent를 사용하는 방법 중 하나로 일반적인 에이전트의 기능을 확장하여, 더 복잡한 문제를 단계적으로 해결할 수 있음
> 추론과 행동을 기반으로 함.
- 사용자 쿼리 수신: 사용자로부터의 질문이나 요청을 받음
- 의도 분류: ReAct 프롬프팅을 사용하여 사용자의 의도를 정확하게 분류
- 외부 API 및 내부 지식 기반 조회: 사용자의 요청에 따라 필요한 정보를 검색하기 위해 외부 서비스나 내부 데이터베이스를 조회
- 적절한 응답 생성: 검색된 정보를 기반으로 사용자의 질문에 대한 적절한 응답을 생성

In [ ]:
tools = load_tools(
    ["llm-math"], 
    llm=llm
)

# add our custom SQL db tool
tools.append(sql_tool)

LangChain docstores 를 이용하여 웹 페이지의 정보 이용 가능 
- Search: 검색
- Lookup: 청크 찾음

In [ ]:
from langchain import Wikipedia
from langchain.agents.react.base import DocstoreExplorer

docstore=DocstoreExplorer(Wikipedia())
tools = [
    Tool(
        name="Search",
        func=docstore.search,
        description='search wikipedia'
    ),
    Tool(
        name="Lookup",
        func=docstore.lookup,
        description='lookup a term in wikipedia'
    )
]

In [ ]:
docstore_agent = initialize_agent(
    tools, 
    llm, 
    agent="react-docstore", 
    verbose=True,
    max_iterations=3
)

In [ ]:
docstore_agent("What were Archimedes' last words?")

#### 검색을 통한 자체 질문 self-ask-with-search
- 최종 답변을 얻기 위해 필요한 만큼 검색을 수행하고 후속 질문을 함
- serp_api_key 사용 (발급 필요)
- 사용자가 질문하면, 이 질문에 답변하기 위해 일어나는 프로세스에 대해 알려줌
  
  > 언어 모델이 어떻게 추론하고 행동하여 답변을 도출하는지를 보여주는 예입니다. 이 과정은 모델이 복잡한 질문에 대한 답변을 도출하기 위해 논리적인 추론 단계를 거치는 것을 포함합니다.
